In [5]:
# pip install langchain huggingface_hub
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
from pymongo import MongoClient
from datetime import datetime
from langchain_huggingface import HuggingFaceEndpoint

In [2]:
# Load environment variables from .env file
load_dotenv()

# MongoDB connection setup
MONGODB_URI = os.getenv("MONGODB_URI")

# Connect to MongoDB
try:
    client = MongoClient(MONGODB_URI)
    # Ping the database to check if the connection is successful
    client.admin.command('ping')
    print("Connected to MongoDB successfully!")
except Exception as e:
    print("Failed to connect to MongoDB:", e)
    exit(1)

# Select the database and collection
db = client["test"]
collection = db["productdata"]


Connected to MongoDB successfully!


In [6]:
gptApiKey = os.getenv("OPENAI_API_KEY")
openai.api_key = gptApiKey

API_URL = "https://api.openai.com/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {gptApiKey}",
    "Content-Type": "application/json"
}

In [7]:

# Fetch products from MongoDB
products = collection.find()

for product in products:
    
    # Skip if descriptions are already created
    if product.get("Descriptions Created") is False:
        print(f"Product with MongoDB ID {product['_id']} already has descriptions created. Skipping...")
        continue

    # Extract product data from MongoDB
    product_data = {
        "name": product.get('product'),
        "colors": product.get('colors', []),
        "sizes": product.get('sizes', []),
        "primary_color": product.get('primary_color', ''),
        "type": product.get('type', ''),
        "gender": product.get('gender', ''),
    }

    print("\nProcessing product:", product_data)

    # Few-shot prompt example
    few_shot_prompt = (
        "Here are the few examples with attributes of a product as input, and the detailed description of the product as the output.\n"
        "You have to generate the description once only, and don't add any examples. You have to simply give the detailed description of the product based on its attributes.\n\n"
        
        "Example 1:\n"
        "Attributes: {'name': 'DAILY CREW NECK TEE', 'colors': ['BLACK', 'DARK OLIVE', 'IVORY'], 'sizes': ['SMALL', 'MEDIUM', 'LARGE', 'X-LARGE', 'XX-LARGE'], 'primary_color': 'DARK OLIVE', 'type': 'T-Shirt', 'gender': 'Men'}\n"
        "Description: A versatile dark olive crew neck tee designed for men, perfect for everyday casual wear. Its understated color pairs well with jeans, chinos, or shorts for a relaxed and stylish look. Ideal for laid-back outings, layering under jackets, or lounging at home. Available in sizes SMALL, MEDIUM, LARGE, X-LARGE, and XX-LARGE, and in colors BLACK, DARK OLIVE, and IVORY\n\n"
        
        "Example 2:\n"
        "Attributes: {'name': 'RIBBED KNIT TROUSER', 'colors': ['NAVY', 'CAMEL', 'BLACK'], 'sizes': ['SMALL', 'MEDIUM', 'LARGE', 'X-LARGE', 'XX-LARGE'], 'primary_color': 'CAMEL', 'type': 'Bottoms', 'gender': 'Men'}\n"
        "Description: Comfortable and versatile camel ribbed knit trousers designed for men, perfect for casual outings or relaxed settings. These trousers combine style and ease, making them ideal for pairing with a simple tee, sweater, or casual shirt. The ribbed texture adds a modern touch, while the camel color complements a variety of outfits. Available in sizes SMALL, MEDIUM, LARGE, X-LARGE, and XX-LARGE, and in colors NAVY, CAMEL, and BLACK\n\n"

        "Example 3:\n"
        "Attributes: {'name': 'LEATHER LACE-UP SNEAKERS', 'colors': ['ALL BLACK', 'OLIVE', 'TAUPE'], 'sizes': ['40', '41', '42', '43', '44', '45', '46'], 'primary_color': 'TAUPE', 'type': 'Sneakers', 'gender': 'Men'}\n"
        "Description: Stylish taupe leather lace-up sneakers designed for men, perfect for casual outings or smart-casual occasions. These sneakers offer a refined yet versatile look, pairing effortlessly with jeans, chinos, or even tailored trousers. Suitable for day-to-day wear or semi-formal settings, they are a comfortable and chic addition to any wardrobe. Available in sizes 40, 41, 42, 43, 44, 45, and 46, and in colors ALL BLACK, OLIVE, and TAUPE\n\n"
        
        "Now, generate a description for the following product, keeping in mind that the language should be direct, with no extra embellishments:\n\n"
    )

    # Create the product description prompt
    prompt = (
        f"Provide a clear, concise product description for the item named '{product_data['name']}', "
        f"a {product_data['primary_color']} {product_data['type']} for {product_data['gender']}. "
        "Only use the information provided in the attributes and do not add unnecessary details about fabric, fit, or qualities not listed in the attributes. "
        "The description should explain how and where the item might be worn based on its style. "
        "Avoid formal or promotional language like 'introducing' or 'order yours today.' "
        "Do add information about the product size and colors in the response if it is given in the input attributes. "
        "Keep the tone straightforward and focus on how the item can be worn. Mention what types of outfits or occasions are suitable, such as pairing it with specific clothing pieces.\n\n"
        f"Attributes: {product_data}\n{few_shot_prompt}"
    )

    # Call OpenAI API to generate the description

    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),
    )
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
     

    output =response.choices[0].message.content.strip()
    print(f"Generated Description for product {product['_id']}:\n", output)

        # Update MongoDB with generated description
    collection.update_one(
        {"_id": product["_id"]},
        {
            "$set": {
                "detailed_description": output,
                "Embeddings Created": True  # Mark this product as processed
            }
        }
    )
    print(f"Product with MongoDB ID {product['_id']} has been updated with a new description.")




Processing product: {'name': 'Spray Wash Tee', 'colors': ['Charcoal'], 'sizes': ['S', 'M', 'L', 'XL', '2XL'], 'primary_color': 'Charcoal', 'type': 'T-Shirt', 'gender': 'Men'}
Generated Description for product 676a38ac63fe6ff887c1d633:
 The Spray Wash Tee in charcoal is a straightforward and stylish t-shirt for men, ideal for casual wear. Its neutral charcoal color makes it a great choice for pairing with jeans, shorts, or joggers, fitting seamlessly into your everyday wardrobe. Whether you're heading out for a relaxed weekend, running errands, or hanging out with friends, this t-shirt offers a laid-back aesthetic. Available in sizes S, M, L, XL, and 2XL.
Product with MongoDB ID 676a38ac63fe6ff887c1d633 has been updated with a new description.

Processing product: {'name': 'High Neck T-Shirt', 'colors': ['White'], 'sizes': ['S', 'M', 'L', 'XL', '2XL'], 'primary_color': 'White', 'type': 'T-Shirt', 'gender': 'Men'}
Generated Description for product 676a38ac63fe6ff887c1d634:
 A classic wh

APIConnectionError: Connection error.